### Franco Sotomayor Casale A00831450
### Alberto Estrada Guerrero A01276671
### Marcelo Márquez Murillo A01720588
### Edgar Alexandro Castillo Palacios A008305568
### Arturo Garza Campuzano A00828096
### Jose Gerardo Cantu Garcia A00830760

### **Imports**

In [173]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random
import time
import datetime

### **Room Matrix**

In [174]:
def get_room(model):
    room = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Robot) and content.grabbed_box:
                room[x][y] = 1
            elif isinstance(content, Robot) and not content.grabbed_box:
                room[x][y] = 2
            elif isinstance(content, Pallet) and content.box_count == 0:
                room[x][y] = 3
            elif isinstance(content, Pallet) and content.box_count == 1:
                room[x][y] = 4
            elif isinstance(content, Pallet) and content.box_count == 2:
                room[x][y] = 5
            elif isinstance(content, Pallet) and content.box_count == 3:
                room[x][y] = 6
            elif isinstance(content, Pallet) and content.box_count == 4:
                room[x][y] = 7
            elif isinstance(content, Pallet) and content.box_count == 5:
                room[x][y] = 8
            elif isinstance(content, Box):
                room[x][y] = 9

    return room

### **Pallet Class**

In [175]:
class Pallet(Agent):
    total_boxes = 0

    def __init__(self, unique_id: int, model: Model):
        super().__init__(unique_id, model)
        self.box_count = 0

### **Box Class**

In [176]:
class Box(Agent):
    def __init__(self, unique_id: int, model: Model):
        super().__init__(unique_id, model)
        self.is_grabbed = False
        self.on_pallet = False
        self.pallet_pos = None

    def grab_box(self):
        self.model.grid.remove_agent(self)

    def place_box(self):
        y, x = self.pallet_pos
        pallet = None
        for content in self.model.grid[y][x]:
            if isinstance(content, Pallet):
                pallet = content
                self.model.grid.remove_agent(content)
                break
        self.model.grid.place_agent(self, self.pallet_pos)
        self.model.grid.place_agent(pallet, self.pallet_pos)
        self.on_pallet = True

### **Robot Class**

In [177]:
class Robot(Agent):
    movements = 0

    def __init__(self, unique_id: int, model: Model):
        super().__init__(unique_id, model)
        self.grabbed_box = False
        self.new_pos = None
        self.box = None

    def step(self):
        neighbors = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True
        )

        for neighbor in neighbors:
            if self.grabbed_box:
                if isinstance(neighbor, Pallet) and neighbor.box_count < 5:
                    self.box.pallet_pos = neighbor.pos
                    neighbor.box_count += 1
                    Pallet.total_boxes += 1
                    self.grabbed_box = False
                    self.box.place_box()
                    self.box = None
                    break
            else:
                if isinstance(neighbor, Box) and not neighbor.is_grabbed:
                    self.grabbed_box = True
                    self.box = neighbor
                    neighbor.is_grabbed = True
                    neighbor.grab_box()
                    break

    def advance(self):
        n_position = None
        found_pos = False

        neighborhood = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )

        while not found_pos: # Mientras que no hemos encontrado una nueva dirección
            if len(neighborhood) != 0:
                n_position = self.random.choice(neighborhood)
                y, x = n_position
                cell = self.model.grid[y][x]
                can_move = True
                for obj in cell:
                    if isinstance(obj, Robot):
                        neighborhood.remove(n_position)
                        can_move = False
                        break
                    elif isinstance(obj, Box) and self.grabbed_box:
                        neighborhood.remove(n_position)
                        can_move = False
                        break
                    elif isinstance(obj, Pallet) and not self.grabbed_box:
                        # print("Encontre tarima pero no tengo caja")
                        neighborhood.remove(n_position)
                        can_move = False
                        break
                    elif isinstance(obj, Pallet) and obj.box_count > 4:
                        neighborhood.remove(n_position)
                        can_move = False
                        break

                if can_move: # Si encontramos dirección vacía o con caja sin agarrar
                    found_pos = True

            else:
                # print("Found no new position")
                n_position = self.pos
                found_pos = True

        self.new_pos = n_position
        self.model.grid.move_agent(self, self.new_pos)
        Robot.movements += 1

### **Room Class**

In [178]:
class Room(Model):
    def __init__(self, width, height, num_boxes, num_pallets):
        self.num_robots = 5
        self.K = num_boxes
        self.num_pallets = num_pallets
        self.grid = MultiGrid(width=width, height=height, torus=False)
        self.schedule = SimultaneousActivation(self)
        self.counter = 0

        # Spawn the robots
        for num in range(self.num_robots):
            self.spawn_obj(Robot, num)

        # Spawn the boxes
        for num in range(self.K):
            self.spawn_obj(Box, num+self.num_robots)

        # Spawn the pallets
        for num in range(self.num_pallets):
            self.spawn_obj(Pallet, num+self.num_robots+self.K)

        self.datacollector = DataCollector(
            model_reporters={'Room': get_room}
        )

    def spawn_obj(self, class_to_use, obj_id):
        empty_cells_list = list(self.grid.empties)
        empty_cell = self.random.choice(empty_cells_list)
        obj_to_spawn = class_to_use(obj_id, self)
        self.schedule.add(obj_to_spawn)
        self.grid.place_agent(obj_to_spawn, empty_cell)

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        self.counter += 1

    def all_done(self):
        if Pallet.total_boxes == self.K:
            return True
        else:
            return False
        # for celda in self.grid.coord_iter():
        #     contenido_celda, x, y = celda
        #     for contenido in contenido_celda:
        #         if isinstance(contenido, Box):
        #             if contenido.on_pallet:
        #                 break
        #             else:
        #                 return False
        # return True


### **Run Model**

In [179]:
WIDTH = 10
HEIGHT = 10

NUM_BOXES = 13
if NUM_BOXES % 5 == 0:
    NUM_PALLETS = int(NUM_BOXES / 5)
else:
    NUM_PALLETS = int(NUM_BOXES / 5) +  1

start_time = time.time()

models = Room(width=WIDTH, height=HEIGHT, num_boxes=NUM_BOXES, num_pallets=NUM_PALLETS)

while not models.all_done():
    models.step()

# Imprimimos el tiempo que le tomo correr al modelo
total_time = str(datetime.timedelta(seconds = (time.time() - start_time)))

### **Visualization**

In [180]:
all_rooms = models.datacollector.get_model_vars_dataframe()

In [181]:
%%capture

fig,axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])

#         Floor,  Robot-box  Robot  Pallet  Pallet1   Pallet2   Pallet3   Pallet4   Pallet5   Box
colors = ["white", "green", "blue", "brown", "pink", "purple", "violet", "magenta", "black", "orange"]
color_map = LinearSegmentedColormap.from_list('', colors, len(colors))

patch = plt.imshow(all_rooms.iloc[0][0], cmap=color_map, vmin=0, vmax=len(colors)-1)

def animate(i):
    patch.set_data(all_rooms.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_rooms))

In [ ]:
anim

In [ ]:
print(f"Número de movimientos realizados por los robots: {Robot.movements}")
print(f"Tiempo para que todas las cajas estén en pilas: {total_time}")

#### **Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Cómo sería? Descríbela.**
Para la elaboración de este código se implementaron métodos de búsqueda sobre un agente hacia sus vecinos o incluso iteraciones sobre la cuadrícula entera; lo anterior con el fin de conseguir información sobre sus celdas de una manera más eficiente.
Por ejemplo, la manera en que corre nuestro código es que se sigue iterando hasta que todas las cajas estén en una de las tarimas. Para revisar esto, originalmente se utilizaba una función la cual iteraba sobre toda la cuadrícula primero iterando sobre la lista que comprende la matriz y, por consiguiente, dentro del conjunto de listas anidadas que conforman la dimensión de esta teniendo por ende una complejidad de O(n^2). No obstante, la lógica implementada en la presente simulación aborda el problema en cuestión guardando en la clase de tarimas una variable la cual se sumaría cada vez se agrega una nueva caja haciendo uso de una complejidad de O(1) por medio de declaraciones "If/Else". Esto ayuda a disminuir la cantidad de tiempo que toma en correr el código.
Una estrategia que se podría implementar para disminuir ambos el tiempo de ejecución y la cantidad total de movimientos realizados sería a través de la implementación de bloqueos de las zonas donde se encuentra una tarima cuya capacidad esté al límite en un alcance de 3x3 celdas donde la tarima se encuentre en el centro de tal manera que un robot no pueda pasar por esa zona y hacer movimientos en vano.
Otra estrategia que se puede llevar a cabo sería hacer los movimientos de los robots aún más inteligentes a través de las siguientes dos maneras: 1) Cuando un robot detecta una en su vecindad, pero dicha caja no es directamente accesible para recoger, que el robot se ponga sobre la caja y agarrarla. 2) Cada instancia del robot tendría una lista de las posiciones de las tarimas y cuando recoge una caja, que identifique la tarima más cercana para llegar a ella y cumplir su tarea.